<a href="https://colab.research.google.com/github/Guru-180188/Projects/blob/main/GAN_MODEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://machinelearningmastery.com/how-to-develop-a-generative-adversarial-network-for-an-mnist-handwritten-digits-from-scratch-in-keras/
# example of training a gan on mnist
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
	model = Sequential()
	model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
	return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

# load and prepare mnist training images
def load_real_samples():
	# load mnist dataset
	(trainX, _), (_, _) = load_data()
	# expand to 3d, e.g. add channels dimension
	X = expand_dims(trainX, axis=-1)
	# convert from unsigned ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [0,1]
	X = X / 255.0
	return X

# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch + 1)
	g_model.save(filename)

# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# create training set for the discriminator
			X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
			# update discriminator model weights
			d_loss, _ = d_model.train_on_batch(X, y)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)


"""

Sure, let's go through the code line by line and explain each part:

```python"""
# Import necessary libraries
from numpy import expand_dims, zeros, ones, vstack, randn, randint
from keras.datasets.mnist import load_data
from keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense, Reshape, Flatten, Conv2D, Conv2DTranspose, LeakyReLU, Dropout
from matplotlib import pyplot
"""

The code begins by importing the required libraries: NumPy for numerical operations, Keras for building and training the neural network models, and Matplotlib for plotting.

```python"""
# Define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model
"""

The `define_discriminator` function creates the discriminator model. It's a convolutional neural network (CNN) that takes a 28x28 grayscale image as input and outputs a binary classification (real or fake). The architecture consists of two convolutional layers with LeakyReLU activation and dropout to prevent overfitting. After flattening, a dense layer with a sigmoid activation is used to predict the binary output. The model is compiled with the Adam optimizer and binary cross-entropy loss.

```python"""
# Define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    n_nodes = 128 * 7 * 7
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    return model
"""

The `define_generator` function creates the generator model. It takes a latent_dim as input, which represents the size of the random noise vector (latent space). The generator starts with a dense layer that transforms the random noise into a higher-dimensional representation. It then reshapes this representation into a 7x7x128 tensor. Two transpose convolutional layers are used to gradually upsample the tensor to 28x28 size, and finally, the generator outputs a 28x28 grayscale image using a sigmoid activation.

```python"""
# Define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

"""

The `define_gan` function creates the GAN model by combining the generator and discriminator. The discriminator's weights are set to non-trainable to avoid updating them during the generator training. The generator is added as the first layer of the GAN, followed by the discriminator. The GAN is compiled with the Adam optimizer and binary cross-entropy loss.

```python"""
# Load and prepare MNIST training images
def load_real_samples():
    (trainX, _), (_, _) = load_data()
    X = expand_dims(trainX, axis=-1)
    X = X.astype('float32')
    X = X / 255.0
    return X
"""

The `load_real_samples` function loads the MNIST dataset and prepares the images. It loads the dataset, expands the dimensions to add a channel dimension (since the original dataset is grayscale), converts pixel values to floating-point numbers, and scales them to the range [0, 1].

(Continued in the next message due to character limit)





Continuing from the previous explanation:

```python"""
# Select real samples
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples, 1))
    return X, y
"""

The `generate_real_samples` function selects a random batch of real images from the dataset. It takes the prepared dataset and the number of samples to generate as input. It randomly selects indices (ix) from the dataset and retrieves the corresponding images (X). For each real image, it assigns a "real" class label (1).

```python"""
# Generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
"""

The `generate_latent_points` function generates random points in the latent space (random noise) as input for the generator. It takes the size of the latent space (latent_dim) and the number of samples to generate (n_samples) as input. It creates a random vector (x_input) of shape (latent_dim * n_samples) using NumPy's `randn` function, and then reshapes it into a batch of inputs for the generator.

```python"""
# Use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    x_input = generate_latent_points(latent_dim, n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples, 1))
    return X, y
"""

The `generate_fake_samples` function uses the generator to produce fake images given random points in the latent space. It takes the generator model, the size of the latent space (latent_dim), and the number of samples to generate (n_samples) as input. It first generates random points in the latent space using `generate_latent_points`. Then, it passes these points through the generator (`g_model.predict`) to produce fake images (X). For each fake image, it assigns a "fake" class label (0).

```python"""
# Create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
    for i in range(n * n):
        pyplot.subplot(n, n, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    pyplot.savefig(filename)
    pyplot.close()
"""

The `save_plot` function creates a grid plot of generated images and saves it as a PNG file. It takes the generated images (examples), the current epoch number (epoch), and the number of images to display in each row/column (n). The function iterates through the generated images, creates a subplot for each image, and sets the axis off. It then displays the image using a reversed grayscale color map (`gray_r`). Finally, it saves the plot to a file named with the epoch number.

```python"""
# Evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    save_plot(x_fake, epoch)
    filename = 'generator_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)
"""

The `summarize_performance` function evaluates the discriminator's performance, plots generated images, and saves the generator model at the specified epoch. It takes the current epoch number (epoch), the generator model (g_model), the discriminator model (d_model), the dataset, the size of the latent space (latent_dim), and the number of samples to generate (n_samples) as input.

The function first generates a batch of real samples and evaluates the discriminator's accuracy on real data. Then, it generates a batch of fake samples using the generator and evaluates the discriminator's accuracy on the fake data. It prints the accuracy of the discriminator on both real and fake samples.

Next, it calls the `save_plot` function to create and save a grid plot of the generated fake images. Finally, it saves the generator model to a file with a name that includes the epoch number.

```python"""
# Train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=25, n_batch=256):
    bat_per_epo = int(dataset.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    for i in range(n_epochs):
        for j in range(bat_per_epo):
            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            d_loss, _ = d_model.train_on_batch(X, y)
            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch, 1))
            g_loss = gan_model.train_on_batch(X_gan, y_gan)
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
        if (i+1) % 10 == 0:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
"""

The `train` function is responsible for training the GAN. It takes the generator model (g_model), discriminator model (d_model), GAN model (gan_model), the dataset, the size of the latent space (latent_dim), the number of epochs to train (n_epochs), and the batch size (n_batch) as input.

The function starts by calculating the number of batches per epoch (bat_per_epo) and half of the batch size (half_batch). Then, it runs a loop for the specified number of epochs.

Within each epoch, it iterates over the batches in the training dataset. For each batch, it generates a batch of real and fake samples using `generate_real_samples` and `generate_fake_samples`. It then stacks the real and fake samples along with their corresponding labels to create a training set for the discriminator.

Next, it trains the discriminator model on the batch of real and fake samples, updating its weights. It then generates a batch of points in the latent space and sets the target labels to "real" (1) to trick the generator.

Finally, it trains the GAN model on the batch of latent space points with the inverted labels, updating the generator's weights. During training,

 it prints the discriminator and generator losses for each batch.

Every 10 epochs, the `summarize_performance` function is called to evaluate the current performance of the GAN, plot generated images, and save the generator model.

Overall, the code implements a GAN for generating handwritten digits similar to those in the MNIST dataset and trains it for a specified number of epochs.
"""

4/4 [==============================] - 0s 6ms/step
>1, 1/234, d=0.708, g=0.756
4/4 [==============================] - 0s 7ms/step
>1, 2/234, d=0.661, g=0.849
4/4 [==============================] - 0s 5ms/step
>1, 3/234, d=0.634, g=0.928
4/4 [==============================] - 0s 6ms/step
>1, 4/234, d=0.620, g=0.921
4/4 [==============================] - 0s 5ms/step
>1, 5/234, d=0.639, g=0.786
4/4 [==============================] - 0s 4ms/step
>1, 6/234, d=0.648, g=0.700
4/4 [==============================] - 0s 7ms/step
>1, 7/234, d=0.622, g=0.691
4/4 [==============================] - 0s 4ms/step
>1, 8/234, d=0.579, g=0.696
4/4 [==============================] - 0s 4ms/step
>1, 9/234, d=0.535, g=0.702
4/4 [==============================] - 0s 4ms/step
>1, 10/234, d=0.495, g=0.706
4/4 [==============================] - 0s 4ms/step
>1, 11/234, d=0.448, g=0.712


KeyboardInterrupt: ignored